In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
need_plyfile=False
need_open3d=False

try:
    from plyfile import PlyData, PlyElement
except ModuleNotFoundError:
    need_plyfile=True
if need_plyfile:
    !pip install plyfile
    from plyfile import PlyData, PlyElement

try:
    import open3d as o3d
except ModuleNotFoundError:
    need_open3d=True
if need_open3d:
    !pip install open3d
    import open3d as o3d

import numpy as np
import os

def load_ply(file_path):
    return PlyData.read(file_path)

def save_ply(vertices, faces, file_path):
    vertex_element = PlyElement.describe(vertices, 'vertex')
    face_element = PlyElement.describe(faces, 'face')
    PlyData([vertex_element, face_element]).write(file_path)

def centralize_model(vertices):
    points = np.vstack([vertices['x'], vertices['y'], vertices['z']]).T
    center = np.mean(points, axis=0)
    centered_points = points - center
    vertices['x'] = centered_points[:, 0]
    vertices['y'] = centered_points[:, 1]
    vertices['z'] = centered_points[:, 2]
    return vertices

def normalize_models(file_names, path):
    output_dir = os.path.join(path ,'/NovosNormalizados/')  # Diretório onde os modelos normalizados serão salvos

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Obter a lista de arquivos .ply na pasta
    file_paths = [os.path.join(path, file_name) for file_name in file_names]

    # Carregar e centralizar todos os modelos
    all_vertices = []
    all_faces = []
    point_clouds = []
    for file_path in file_paths:
        ply_data = load_ply(file_path)
        vertices = ply_data['vertex']
        faces = ply_data['face']

        # Centralizar o modelo
        vertices = centralize_model(vertices)

        # Converter os vértices para um array NumPy
        vertices_np = np.vstack([vertices['x'], vertices['y'], vertices['z']]).T

        all_vertices.append(vertices_np) # Armazenar o array NumPy
        all_faces.append(faces)

    # Calcular os limites globais dos modelos centralizados
    all_points = np.concatenate(all_vertices, axis=0) # Concatenar os arrays NumPy
    min_bound = np.min(all_points, axis=0)
    max_bound = np.max(all_points, axis=0)

    # Calcular o centro global e o maior comprimento de lado da bounding box global
    global_center = (min_bound + max_bound) / 2
    scale = np.max(max_bound - min_bound)

    # Normalizar cada modelo
    for i, (vertices, faces) in enumerate(zip(all_vertices, all_faces)): # Iterar sobre os arrays NumPy
        # Centralizar e escalar
        normalized_points = (vertices - global_center) / scale

        # Criar um novo PlyElement a partir dos vértices normalizados
        normalized_vertices = np.empty(normalized_points.shape[0], dtype=[('x', 'f4'), ('y', 'f4'), ('z', 'f4')])
        normalized_vertices['x'] = normalized_points[:, 0]
        normalized_vertices['y'] = normalized_points[:, 1]
        normalized_vertices['z'] = normalized_points[:, 2]


        # Criar novos elementos PlyElement
        vertex_element = PlyElement.describe(normalized_vertices, 'vertex') # Passar o array NumPy

        # Salvar a nova subdivisão em um arquivo .ply
        output_file = os.path.join(output_dir, f'{file_names[i]}')
        PlyData([vertex_element, faces]).write(output_file)



In [ ]:
# Exemplo de uso
files = [
    'Homo erectus.ply',  # Caminho para o primeiro arquivo .ply
    'Homo neanderthalensis.ply',  # Caminho para o segundo arquivo .ply
    'Homo sapiens.ply'   # Caminho para o terceiro arquivo .ply
]

path = '/content/drive/MyDrive/TCC/Modelos/'
normalize_models(files, path)